# Quick look at a variable
Edit this later!

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr

from dask.distributed import Client
import glob
import catalog
import util
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import pop_tools

xr.set_options(keep_attrs=True)

## Parameters

In [ ]:
variable_id = 'TEMP'

### Connect to cluster

In [ ]:
cluster, client = util.get_ClusterClient()
cluster.scale(12)

## Load the data

In [ ]:
case = 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.002branch'
path = '/glade/campaign/cesm/development/bgcwg/projects/CESM2-OMIP2-like-4p2z-run/'+case+'/ocn/proc/tseries/month_1'
coords = {'x':'TLONG','y':'TLAT'}
path

In [ ]:
files = glob.glob(f'{path}/{case}.pop.h.*.195801-202112.nc')
ds=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True,
                       drop_variables=["transport_components", "transport_regions", 'moc_components'], decode_times=True)
ds

In [ ]:
# dt_wgt = ds.time_bound.diff('d2').squeeze()
# dt_wgt /= dt_wgt.sum()
# dt_wgt

In [ ]:
# ds[variable_id].weighted(dt_wgt).mean('time').isel(z_t=0).plot()
ds[variable_id].mean('time').isel(z_t=0).plot()

In [ ]:
ds_grid = pop_tools.get_grid('POP_gx1v7')
lons = ds_grid.TLONG
lats = ds_grid.TLAT
depths = ds_grid.z_t * 0.01

# FIX LABELS + COLOR ON THIS PLOT

In [ ]:
fig = plt.figure(figsize=(8,6))

ax = fig.add_subplot(1,1,1, projection=ccrs.Robinson(central_longitude=305.0))
land = cartopy.feature.NaturalEarthFeature('physical', 'land', scale='110m', edgecolor='k', facecolor='white', linewidth=0.5)
ax.add_feature(land)
ax.set_title('CESM surface ' + variable_id, fontsize=10)
lon, lat, field = util.adjust_pop_grid(lons, lats, ds[variable_id].mean('time').isel(z_t=0))
pc1=ax.pcolormesh(lon, lat,field, vmin=0, vmax=20, cmap='Reds',
                 transform=ccrs.PlateCarree())
cbar1 = fig.colorbar(pc1, ax=ax,extend='max',label=ds[variable_id].units)